In [19]:
import json
import pandas as pd
import cmath
import xml.etree.ElementTree as et
import numpy as np
from transformations import *
from datetime import datetime, timedelta


In [20]:
global keeperA
global keeperB
global teamA
global teamB
global current_possession,poss_types, withshot,withshotongoal, withgoal, flank

In [21]:
pd.set_option('display.max_columns', 5000)
#pd.set_option('display.max_rows', None)

In [22]:
#instatiate empty dataframe with wyscout format
template_path='template.csv'
df=pd.read_csv(template_path)
wyscoutdf=df

In [23]:
#set all unknown attributes
id=videoTimestamp=relatedEventId=teamId=opponentTeamId=playerId=passrecipientId=possessionId = 0
possessioneventIndex=possessionteamId=groundDuelopponentId=groundDuelrelatedDuelId=infractionopponentId=shotgoalkeeperActionId=shotgoalkeeperId=aerialDuelopponentId=aerialDuelrelatedDuelId = 0
passheight =groundduel_take_on= ground_side=None
postshotxg=shotxg=shotgoalzone=possessionattackxg=aerialduelheight=aerialFirsttouch=aerialdueloppheight=np.nan


In [24]:
#to do
carry_x=carry_y=carry_prog=possessioneventsNumber=np.nan

In [34]:
#load instat file
instat_path = 'data/National_Men_2022-03-29_UEFA U21 Championship Qualification_Netherlands U21_vs_Switzerland U21_1904894_2022-03-29_Netherlands U21_vs_Switzerland U21_1904894_rawdata.xml'
instatdf=pd.read_xml(instat_path)
matchId=getmatchId(instatdf)
instatdf=instatdf.iloc[2:,:]
instatdf.drop(columns=['column','number','dl', 'uid', 'action_id', 'player_id', 'team_id', 'standart_id', 'ts', 'position_id', 'opponent_id', 'opponent_team_id','opponent_position_id', 'zone_id', 'zone_dest_id', 'possession_id', 'possession_team_id', 'possession_team_id', 'attack_status_id', 'attack_type_id', 'attack_team_id', 'body_id' ], inplace=True)
#get the goalkeepers
keeperA,keeperB= get_keepers(instatdf)



In [ ]:
instatdf

In [35]:
#remove unnecessairy rows
mask = (instatdf['second']==0.00) & (~instatdf['player_name'].isna()) & (instatdf['action_name']!='Attacking pass accurate')&(instatdf['action_name']!='Non attacking pass accurate')&(instatdf.index<100)
instatdf = instatdf[~mask]
todrop = ['line-up', 'Substitute player', '1st half','2nd half', 'Players, that created offside trap']
mask = ~instatdf['action_name'].isin(todrop)
instatdf = instatdf[mask]
teamA,teamB=getformations(instatdf)
teamA, teamB

(['Switzerland U21', '4-3-1-2'], ['Netherlands U21', '4-4-2'])

In [37]:
instatdf=instatdf.reset_index()

In [ ]:
instatdf.iloc[:50]

In [39]:
mask = ((instatdf['second']==0)&(instatdf['action_name']!='Attacking pass accurate')
        &(instatdf['action_name']!='Non attacking pass accurate')
        &(instatdf['action_name']!='Non attacking pass inaccurate')&(instatdf['action_name']!='Attacking pass inaccurate')&(instatdf.index<200))
instatdf=instatdf[~mask]
#ready for wyscout transformation

In [ ]:
instatdf

In [13]:
instatdf.iloc[2484:]

,index,id,action_name,player_name,team_name,half,second,standart_name,position_name,opponent_name,opponent_team_name,opponent_position_name,zone_name,zone_dest_name,pos_x005F_x,pos_y,pos_dest_x005F_x,pos_dest_y,len,possession_name,possession_team_name,possession_time,possession_number,attack_status_name,attack_type_name,attack_flang_id,attack_flang_name,attack_team_name,attack_number,direction,body_name,gate_x005F_x,gate_y,assistant_id,assistant_name,shot_type,touches,shot_handling
2577,2688,45946.0,Attacking pass accurate,Cheveyo Tsawa,Switzerland U16,2.0,2944.37,Open play,Midfielder - Right,Mario Greco,Switzerland U16,None,4R,4B,94.0,11.1,100.2,30.9,20.7,Ball possession,Switzerland U16,0.0,235.0,Attack,corner attack,NaN,None,Switzerland U16,163.0,NaN,Right foot,NaN,NaN,NaN,None,None,None,None
2578,2689,45947.0,Crosses accurate,Cheveyo Tsawa,Switzerland U16,2.0,2944.37,Open play,Midfielder - Right,Mario Greco,Switzerland U16,None,4R,4B,94.0,11.1,100.2,30.9,NaN,Ball possession,Switzerland U16,0.0,235.0,Attack,corner attack,NaN,None,Switzerland U16,163.0,NaN,Right foot,NaN,NaN,NaN,None,None,None,None
2579,2690,47625.0,Accurate crossing from set piece,Cheveyo Tsawa,Switzerland U16,2.0,2944.37,Open play,Midfielder - Right,Mario Greco,Switzerland U16,None,CrossRCor,CrossNearPost,94.0,11.1,100.2,30.9,NaN,None,Switzerland U16,NaN,235.0,None,corner attack,NaN,None,Switzerland U16,163.0,NaN,None,NaN,NaN,NaN,None,None,None,None
2580,2691,45948.0,Air challenge,Miguel Angel Santaella Lara,Spain U16,2.0,2945.64,Open play,Defender - Left central,Mario Greco,Switzerland U16,None,1B,None,4.8,37.1,NaN,NaN,NaN,End,Switzerland U16,4.0,235.0,End,corner attack,NaN,None,Switzerland U16,163.0,NaN,None,NaN,NaN,NaN,None,None,None,None
2581,2692,45949.0,Match end,None,None,2.0,2949.40,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None


In [25]:
test=get_primary_type(instatdf,265)
print(test)

None


In [18]:
instatdf.iloc[265:300]

,index,id,action_name,player_name,team_name,half,second,standart_name,position_name,opponent_name,opponent_team_name,opponent_position_name,zone_name,zone_dest_name,pos_x005F_x,pos_y,pos_dest_x005F_x,pos_dest_y,len,possession_name,possession_team_name,possession_time,possession_number,direction,attack_status_name,attack_type_name,attack_flang_id,attack_flang_name,attack_team_name,attack_number,gate_x005F_x,gate_y,body_name,shot_type,touches,shot_handling,assistant_id,assistant_name
320,417,18217.0,Dribbling,Maeva Jocelyne Clemaron,Servette Chenois,1.0,622.07,Open play,Defensive midfielder - Left central,None,None,None,2L,None,28.9,48.9,51.9,54.0,23.6,Ball possession,Servette Chenois,0.0,31.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,19.0,NaN,NaN,None,None,None,None,NaN,None
321,418,15282.0,Successful dribbling,Maeva Jocelyne Clemaron,Servette Chenois,1.0,623.67,Open play,Defensive midfielder - Left central,Fabienne Valerie Humm,Zurich,Forward - Right central,2L,None,38.7,50.9,NaN,NaN,NaN,Ball possession,Servette Chenois,0.0,31.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,19.0,NaN,NaN,None,None,None,None,NaN,None
322,419,15283.0,Attacking pass accurate,Maeva Jocelyne Clemaron,Servette Chenois,1.0,625.64,Open play,Defensive midfielder - Left central,Agata Filipa Pinto Pinto Coelho Pimenta,Servette Chenois,Defender - Left,2L,3L,51.9,54.0,63.4,66.4,16.9,Ball possession,Servette Chenois,0.0,31.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,19.0,NaN,NaN,None,None,None,None,NaN,None
323,420,18677.0,End of unsuccessful team pressing,None,Zurich,1.0,625.64,Open play,None,None,None,None,3R,None,53.1,14.0,NaN,NaN,NaN,Ball possession,Servette Chenois,NaN,31.0,NaN,None,Positional attack,1.0,left flank,Servette Chenois,19.0,NaN,NaN,None,None,None,None,NaN,None
324,421,15284.0,Ball receiving,Agata Filipa Pinto Pinto Coelho Pimenta,Servette Chenois,1.0,626.62,Open play,Defender - Left,None,None,None,3L,None,63.4,66.4,NaN,NaN,NaN,Ball possession,Servette Chenois,0.0,31.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,19.0,NaN,NaN,None,None,None,None,NaN,None
325,422,15285.0,Attacking pass accurate,Agata Filipa Pinto Pinto Coelho Pimenta,Servette Chenois,1.0,628.77,Open play,Defender - Left,Natalia Alessandra Padilla-Bidas,Servette Chenois,Attacking midfielder - Left,3L,3L,69.0,65.2,73.1,54.8,11.2,Ball possession,Servette Chenois,0.0,31.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,19.0,NaN,NaN,None,None,None,None,NaN,None
326,423,15286.0,Ball receiving,Natalia Alessandra Padilla-Bidas,Servette Chenois,1.0,629.51,Open play,Attacking midfielder - Left,None,None,None,3L,None,73.1,54.8,NaN,NaN,NaN,Ball possession,Servette Chenois,0.0,31.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,19.0,NaN,NaN,None,None,None,None,NaN,None
327,424,15287.0,Attacking pass inaccurate,Natalia Alessandra Padilla-Bidas,Servette Chenois,1.0,631.54,Open play,Attacking midfielder - Left,Viktoria Pinther,Zurich,Midfielder - Right,3L,3L,72.4,56.5,75.3,60.4,4.9,Ball possession,Servette Chenois,0.0,31.0,NaN,Attack,Positional attack,1.0,left flank,Servette Chenois,19.0,NaN,NaN,None,None,None,None,NaN,None
328,425,18365.0,Lost balls,Natalia Alessandra Padilla-Bidas,Servette Chenois,1.0,631.54,Open play,None,None,None,None,3L,None,75.3,60.4,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
329,426,15288.0,Ball receiving,Viktoria Pinther,Zurich,1.0,632.01,Open play,Midfielder - Right,None,None,None,2R,None,29.7,7.6,NaN,NaN,NaN,Transition of possession,Servette Chenois,53.0,31.0,NaN,End,Positional attack,1.0,left flank,Servette Chenois,19.0,NaN,NaN,None,None,None,None,NaN,None


In [12]:
a,b,c

(2609, 'postmatch_penalty', ['touch_in_box', 'loss'])

In [ ]:
row=instatdf.iloc[2468]
print(row)

In [21]:
#instatiate first possession
current_possession,poss_types, withshot,withshotongoal, withgoal, flank=setnewpossession(instatdf,0) #start x, y, end x, y, possession_duration, possessionstart index|possession types, attack variables


In [24]:
def create_event(instat, ind):
    global current_possession,poss_types, withshot,withshotongoal, withgoal, flank
    index_instat, typeprimary, typesecondary = get_event_type(instat, ind)
    action = instat['action_name'].iloc[ind]
    #setup every attribute
    period=get_period(instat,ind)
    minute, second, matchTimestamp = get_time(instat, ind, period)
    locx, locy = get_location(instat, ind)
    teamname = instat['team_name'].iloc[ind]
    teamformation = teamA[1] if (teamname==teamA[0]) else teamB[1]
    oppteamname = teamB[0] if (teamname==teamA[0]) else teamA[0]
    oppteamformation = teamB[1] if (teamname==teamA[0]) else teamA[1]
    playername = instat['player_name'].iloc[ind]  #have to change to player names
    playerposition = position_transform(instat['position_name'].iloc[ind], teamformation)
    passlength = instat['len'].iloc[ind]
    isnewposs,possindex = newposs(instat, ind, index_instat)
    possteamname = instat['possession_team_name'].iloc[ind]
    possteamformation = teamA[1] if (possteamname==teamA[0]) else teamB[1]
    possession = instat['possession_name'].iloc[ind]
    keptPoss=None if ('defensive_duel' in typesecondary) else (False if possession=='Transition of possession' else True)
    stopped_prog=None if ('offensive_duel' in typesecondary) else (True if possession=='Transition of possession' else False)
    keepercoord_x, keepercoord_y=get_goalkeeper_coordinates(instat, ind, index_instat, keeperA, keeperB)
    reflexsave=False

    #passing
    if not (pd.isna(instat["pos_dest_x005F_x"].iloc[ind])):
        passaccurate=isaccurate_pass(action)
        passangle=calculate_angle(instat["pos_x005F_x"].iloc[ind], instat["pos_y"].iloc[ind], instat["pos_dest_x005F_x"].iloc[ind], instat["pos_dest_y"].iloc[ind])
        passend_x, passend_y=get_dest_location(instat,ind)
        if(passaccurate):
            passrec, passrec_pos_instat = get_pass_recipient(instat, ind)
            passrec_pos = position_transform(passrec_pos_instat, teamformation) 
        else:passrec= passrec_pos_instat=passrec_pos=np.nan
    else:
        passaccurate=passangle=passend_x=passend_y=passrec=passrec_pos=np.nan
    
    #shot
    if (isshot(action)):
        goalkeeper = keeperA[0] if (teamname!=keeperA[1]) else keeperB[0]
        bodypart=bodypart_transform(instat['body_name'].iloc[ind])
        isgoal= True if 'goal' in typesecondary else False
        isontarget= True if (action=='Shot on target' or action == 'Goal') else False
    else:
        bodypart=isgoal=isontarget=goalkeeper=np.nan

    #infraction
    if(action=='infraction'):
        isyellow=True if ('yellow_card' in typesecondary) else False
        isred = True if ('red_card' in typesecondary) else False
        infraction_type = 'regular_foul' if action!='Deferred foul' else 'late_card_foul' #there are more types of fouls
        infractionopp = instat['opponent_name'].iloc[ind]
        infractionopp_pos = position_transform(instat['opponent_position_name'].iloc[ind], oppteamformation)
    else:
        isyellow=isred=infraction_type=infractionopp=infractionopp_pos=np.nan

    #duel
    if(typeprimary=='duel'):
        aerialopp=aerialopp_pos=groundopp=groundopp_pos =grounddueltype=np.nan 
        ground_stopped_prog=ground_recov_poss =ground_side=groundkeptposs=groundduel_progressed_with_ball=np.nan

        duelopp = instat['opponent_name'].iloc[ind]
        duelopp_pos = position_transform(instat['opponent_position_name'].iloc[ind], oppteamformation)

        #adjust loss tag
        if ('loss' in typesecondary):
            if(stopped_prog==True):
                typesecondary.remove('loss')

        if ('aerial_duel' in typesecondary):
            aerialopp=duelopp
            aerialopp_pos = duelopp_pos
            
        else:
            
            groundopp=duelopp
            groundopp_pos = duelopp_pos
            grounddueltype = 'dribble' if 'dribble' in typesecondary else ('defensive_duel' if 'defensive_duel' in typesecondary else 'offensive_duel')
            groundkeptposs = keptPoss
            groundduel_progressed_with_ball=groundkeptposs #not so clear what progressed with ball means exactly
            ground_stopped_prog=stopped_prog
            ground_recov_poss =ground_stopped_prog #not so clear
            
    else:
        aerialopp=aerialopp_pos=groundopp=groundopp_pos =grounddueltype=np.nan 
        ground_stopped_prog=ground_recov_poss =ground_side=groundkeptposs=groundduel_progressed_with_ball=np.nan

    #possession
    if (current_possession[5]>ind):
        poss_x=poss_y=endposs_x=endposs_y=possessionType=np.nan
    else:
        poss_x=current_possession[0]
        poss_y=current_possession[1]
        endposs_x=current_possession[2]
        endposs_y=current_possession[3]
        possessionType=poss_types
    
    if (isnewposs==True):
        current_possession,poss_types, withshot,withshotongoal, withgoal, flank = setnewpossession(instat, possindex+1) 

    new_event = {
        "id": id,
        "matchId": matchId,
        "matchPeriod":get_period(instat,ind),
        "minute":minute,
        "second":second,
        "matchTimestamp":matchTimestamp,
        "videoTimestamp":videoTimestamp,
        "relatedEventId":relatedEventId,
        "shot":np.nan,
        "groundDuel":np.nan,
        "aerialDuel":np.nan,
        "infraction":np.nan,
        "carry":np.nan,
        "type.primary":typeprimary,	
        "type.secondary":typesecondary,
        "location.x":locx,	
        "location.y":locy,	
        "team.id":teamId,	
        "team.name": teamname,	
        "team.formation":teamformation,	
        "opponentTeam.id":opponentTeamId,
        "opponentTeam.name":oppteamname,	
        "opponentTeam.formation":oppteamformation,	
        "player.id":playerId,	
        "player.name":playername,	
        "player.position":playerposition,	
        "pass.accurate":passaccurate,	
        "pass.angle":passangle,	
        "pass.height":passheight,	
        "pass.length":passlength,	
        "pass.recipient.id":passrecipientId,	
        "pass.recipient.name":passrec,	
        "pass.recipient.position":passrec_pos,
        "pass.endLocation.x":passend_x,	
        "pass.endLocation.y":passend_y,	
        "possession.id":possessionId,	
        "possession.duration":current_possession[4],	
        "possession.types":possessionType,	
        "possession.eventsNumber":possessioneventsNumber,	
        "possession.eventIndex":possessioneventIndex,	
        "possession.startLocation.x":poss_x,
        "possession.startLocation.y":poss_y,	
        "possession.endLocation.x":endposs_x,	
        "possession.endLocation.y":endposs_y,	
        "possession.team.id":possessionteamId,	
        "possession.team.name":possteamname,	
        "possession.team.formation":possteamformation,	
        "possession.attack":np.nan,	
        "pass":np.nan,
        "groundDuel.opponent.id":groundDuelopponentId,	
        "groundDuel.opponent.name":groundopp,	
        "groundDuel.opponent.position":groundopp_pos,	
        "groundDuel.duelType":grounddueltype,	
        "groundDuel.keptPossession":groundkeptposs,	
        "groundDuel.progressedWithBall":groundduel_progressed_with_ball,	
        "groundDuel.stoppedProgress":ground_stopped_prog,	
        "groundDuel.recoveredPossession":ground_recov_poss,	
        "groundDuel.takeOn":groundduel_take_on,	
        "groundDuel.side":ground_side,
        "groundDuel.relatedDuelId":groundDuelrelatedDuelId,	
        "infraction.yellowCard":isyellow,	
        "infraction.redCard":isred,	
        "infraction.type":infraction_type,	
        "infraction.opponent.id":infractionopponentId,	
        "infraction.opponent.name":infractionopp,	
        "infraction.opponent.position":infractionopp_pos,	
        "possession.attack.withShot":withshot,	
        "possession.attack.withShotOnGoal":withshotongoal,	
        "possession.attack.withGoal":withgoal,	
        "possession.attack.flank":flank,	
        "possession.attack.xg":possessionattackxg,	
        "carry.progression":carry_prog,	
        "carry.endLocation.x":carry_x,	
        "carry.endLocation.y":carry_y,	
        "shot.bodyPart":bodypart,	
        "shot.isGoal":isgoal,	
        "shot.onTarget":isontarget,	
        "shot.goalZone":shotgoalzone,	
        "shot.xg": shotxg,	
        "shot.postShotXg":postshotxg,	
        "shot.goalkeeperActionId":shotgoalkeeperActionId,	
        "shot.goalkeeper.id":shotgoalkeeperId,	
        "shot.goalkeeper.name":goalkeeper,	
        "possession":np.nan,	
        "aerialDuel.opponent.id":aerialDuelopponentId,	
        "aerialDuel.opponent.name":aerialopp,	
        "aerialDuel.opponent.position":aerialopp_pos,	
        "aerialDuel.opponent.height":aerialdueloppheight,	
        "aerialDuel.firstTouch":aerialFirsttouch,	
        "aerialDuel.height":aerialduelheight,	
        "aerialDuel.relatedDuelId":aerialDuelrelatedDuelId,	
        "shot.goalkeeper":np.nan,	
        "infraction.opponent":np.nan,	
        "location":np.nan
    }
    
    #check when second event has to be generated
    if (typeprimary=='duel'):
        new_event_2=create_second_duel_event(new_event, keptPoss, stopped_prog, current_possession,poss_types, withshot,withshotongoal, withgoal, flank, isnewposs,ind)
        return index_instat,[new_event, new_event_2]
    elif (action=='Shot on target' or action=='Goal'):
        new_event_2=create_second_shot_event(new_event, keeperA, keeperB, keepercoord_x, keepercoord_y, reflexsave)
        return index_instat,[new_event, new_event_2]
     
    return index_instat, [new_event]

In [23]:
index = 0   
while (True):
    index, newevents = create_event(instatdf, index)
    for event in newevents:
        wyscoutdf = pd.concat([wyscoutdf, pd.DataFrame([event])], ignore_index=True)
    if(instatdf['action_name'].iloc[index]=='Match end'):
        break

C:\Users\Rui\AppData\Local\Temp\ipykernel_38484\2402195945.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  wyscoutdf = pd.concat([wyscoutdf, pd.DataFrame([event])], ignore_index=True)


TypeError: create_second_shot_event() missing 1 required positional argument: 'relfexsave'

In [ ]:
instatdf